In [1]:
import warnings

warnings.filterwarnings("ignore")

from typing import List, Optional, Union

import numpy as np
import pandas as pd

from tsururu.dataset import IndexSlicer, Pipeline, TSDataset
from tsururu.models import CatBoostRegressor_CV
from tsururu.strategies import (
    DirectStrategy,
    FlatWideMIMOStrategy,
    MIMOStrategy,
    RecursiveStrategy,
)
from tsururu.transformers import (
    DateSeasonsGenerator,
    DifferenceNormalizer,
    LagTransformer,
    LastKnownNormalizer,
    SequentialTransformer,
    StandardScalerTransformer,
    TargetGenerator,
    UnionTransformer,
)

In [2]:
def get_results(
    cv: int,
    regime: str,
    y_true: Optional[List[np.ndarray]] = None,
    y_pred: Optional[List[np.ndarray]] = None,
    ids: Optional[List[Union[float, str]]] = None,
) -> pd.DataFrame:
    def _get_fold_value(
        value: Optional[Union[float, np.ndarray]], idx: int
    ) -> List[Optional[Union[float, np.ndarray]]]:
        if value is None:
            return [None]
        if isinstance(value[idx], float):
            return value[idx]
        if isinstance(value[idx], np.ndarray):
            return value[idx].reshape(-1)
        raise TypeError(f"Unexpected value type. Value: {value}")

    df_res_dict = {}

    for idx_fold in range(cv):
        # Fill df_res_dict
        for name, value in [("y_true", y_true), ("y_pred", y_pred)]:
            df_res_dict[f"{name}_{idx_fold+1}"] = _get_fold_value(
                value, idx_fold
            )
        if regime != "local":
            df_res_dict[f"id_{idx_fold+1}"] = _get_fold_value(ids, idx_fold)

    # Save datasets to specified directory
    df_res = pd.DataFrame(df_res_dict)
    return df_res

There are several main objects to look out for when working with the library:
1) `TSDataset`.
2) `Pipeline` and `Transformers`
3) `Strategy`.
4) `Model`.

### TSDataset

This class is needed to store data and meta-information about it.

To initialise it is necessary to submit the data in `pd.DataFrame` format and define some meta-information about roles that necessary for solving the task of time series forecasting: `id`, `date`, `target`.

In [3]:
df_path = "datasets/global/simulated_data_to_check.csv"

dataset_params = {
    "target": {
        "columns": ["value"],
        "type": "continious",
    },
    "date": {
        "columns": ["date"],
        "type": "datetime",
    },
    "id": {
        "columns": ["id"],
        "type": "categorical",
    }
}

In [4]:
dataset = TSDataset(
    data=pd.read_csv(df_path),
    columns_params=dataset_params,
)

freq: Day; period: 1


### Pipeline and Transformers

#### What kind of transformers are there?

Special attention should be paid to the `Transformer` class: the elements of the pipeline that are responsible for transforming the values of a series and generating features. `Pipeline` class is a wrapper over transformers which is needed to provide some additional methods and functions above them.

There are two types of transformers that are used to collect pipelines:
- `Union` transformers;
- `Sequential` transformers.

Below is a list of available Transformers: 
- `StandardScalerTransformer` *(Series4Series)*.
- `DifferenceNormalizer` *(Series4Series)*: subtract the previous value or divide by it.
- `LastKnownNormalizer` *(Features4Features)*: normalize all lags by the last known one: divide by it or subtract.

This three transformers provide flags `transform_features` / `transform_target`, that allow you to manipulate traits and targets separately and get different results from them.

Besides, __DifferenceNormalizer__ and __LastKnownNormalizer__ can be applied in two regimes: `delta` and `ratio`: in the first case, normalisation means subtracting the target value from the current value, and in the second, dividing by it.

- `LabelEncodingTransformer` and `OneHotEncodingTransformer` *(Series4Series)* - encoders for categorical features.
- `TimeToNumGenerator` and `DateSeasonsGenerator` *(Series4Series)* - generator for seasonal features by dates.
- `LagTransformer` *(Series4Features) - generator for lags. 

__!!!The lag transformer must necessarily be present in the sequential transformer, otherwise the features will not be generated.!!!__

Finally, to generate targets, you need to use `TargetGenerator`.

#### Transformers must be assembled in order!

The __SeriesToSeries__ transformers should come first, followed by the LagTransformer and TargetGenerator (__SeriesToFeatures__), and then the __FeaturesToFeatures__ transformers.

#### How to build a Pipeline?

So, there are two ways to build a pipline from transformers: initialise the transformers of interest by hand or use a config in the form of a dictionary. Let's look at both ways.

In [5]:
standard_scaler = StandardScalerTransformer(
    transform_features=True,
    transform_target=True
)

lag = LagTransformer(lags=3)
date_lag = LagTransformer(lags=3)
id_lag = LagTransformer(lags=1)

target_generator = TargetGenerator()

date_seasons = DateSeasonsGenerator(
    seasonalities=["doy", "m", "wd"],
    from_target_date=True,
)

In [6]:
union_1 = UnionTransformer(transformers_list=[lag, target_generator])

seq_1 = SequentialTransformer(transformers_list=[standard_scaler, union_1], input_features=["value"])
seq_2 = SequentialTransformer(transformers_list=[date_seasons, date_lag], input_features=["date"])
seq_3 = SequentialTransformer(transformers_list=[id_lag], input_features=["id"])

union = UnionTransformer(transformers_list=[seq_1, seq_2, seq_3])

In [7]:
pipeline_1 = Pipeline(union, multivariate=False)

In [8]:
pipeline_1.__dict__

{'transformers': <tsururu.transformers.base.UnionTransformer at 0x7f0a4938c610>,
 'multivariate': False,
 'is_fitted': False,
 'strategy_name': None,
 'output_features': None,
 'y_original_shape': None}

Or:

In [9]:
pipeline_params = {
    "target": {
        "columns": ["value"],
        "features": {
            "StandardScalerTransformer":
                {
                    "transform_target": True, 
                    "transform_features": True
                },
            "LagTransformer": {"lags": 7},
        },
    },
    "date": {
        "columns": ["date"],
        "features": {
            "DateSeasonsGenerator": {
                # Use seasonality features from the date column as 
                # features with datetime lags
                # Possible values: [
                #    "y": year, "m": month, "d": day, 
                #    "wd": weekday, "doy": dayofyear,
                #    "hour": hour, "min": minute, "sec": second, 
                #    "ms": microsecond,  "ns": nanosecond
                # ]
                "seasonalities": ['doy', 'm', 'wd'], 
                # Use date from target point to make datetime features
                "from_target_date": True,
            },
            "LagTransformer": {"lags": 3}
        },
    },
    "id": {
        "columns": ["id"],
        "features": {
            "LagTransformer": {"lags": 1},
        },
    }
}

In [10]:
pipeline = Pipeline.from_dict(pipeline_params, multivariate=False)

#### Can I use exogenous variables in the pipeline?

Yes! Exogenous variables can also be specified here. Just include them in your pipeline.

However, their operation is currently tested only for the `MIMOStrategy` in global-modelling. For other strategies support of additional variables is under development.

In [11]:
# pipeline_params["exog_group_1"] = {
#     "columns": ["value"],
#     "features": {
#         "StandardScalerTransformer":
#             {
#                 "transform_target": False, 
#                 "transform_features": True
#             },
#         "LagTransformer": {"lags": 7},
#     },
# }

__Make sure you have the transform_target = False flag for exogenous features!__

#### Model

The model is separate from the strategy. Any model can be run in any strategy if it supports this input and output format

One of the easiest options – is to use GBM.

In [12]:
# Configure the model parameters
model_params = {
    "loss_function": "MultiRMSE",
    "early_stopping_rounds": 100,
    "verbose": 500,
}

# Configure the validation parameters
validation_params = {
    "type": 'KFold',
    "n_splits": 2,
}

In [13]:
model = CatBoostRegressor_CV(validation_params, model_params)

#### Strategy

- _Recursive:_ 
    - one model for all points of the forecast horizon;
    - *training*: the model is trained to predict one point ahead;
    - *prediction*: a prediction is iteratively made one point ahead, and then this prediction is used to further shape the features in the test data. 
- _Recursive-reduced:_
    - one model for all points in the prediction horizon;
    - *training*: the model is trained to predict one point ahead;
    - *prediction*: features are generated for all test observations at once, unavailable values are replaced by NaN.
- _Direct:_ 
    - individual models for each point in the prediction horizon. 
- _MultiOutput (MIMO - Multi-input-multi-output):_
    - one model that learns to predict the entire prediction horizon. 
- _FlatWideMIMO:_.
    - mixture of Direct and MIMO, fit one model, but uses deployed over horizon Direct's features.

In [14]:
horizon = 3
history = 7
step = 1

strategy = RecursiveStrategy(horizon, history, step, model, pipeline)

In [15]:
fit_time, _ = strategy.fit(dataset)

0:	learn: 0.9606080	test: 0.9667407	best: 0.9667407 (0)	total: 60.3ms	remaining: 1m


500:	learn: 0.0051947	test: 0.0053699	best: 0.0053699 (500)	total: 1.34s	remaining: 1.34s
999:	learn: 0.0031608	test: 0.0033676	best: 0.0033676 (999)	total: 2.62s	remaining: 0us

bestTest = 0.003367620955
bestIteration = 999

Fold 0:
MultiRMSE: 0.0033676209549416128
0:	learn: 0.9659554	test: 0.9614093	best: 0.9614093 (0)	total: 5.14ms	remaining: 5.13s
500:	learn: 0.0052698	test: 0.0054766	best: 0.0054766 (500)	total: 1.25s	remaining: 1.25s
999:	learn: 0.0031515	test: 0.0033391	best: 0.0033391 (999)	total: 2.53s	remaining: 0us

bestTest = 0.003339095317
bestIteration = 999

Fold 1:
MultiRMSE: 0.0033390953168007786
Mean MultiRMSE: 0.0034
Std: 0.0


In [16]:
forecast_time, current_pred = strategy.predict(dataset)

freq: Day; period: 1


In [17]:
current_pred

,id,date,value
0,0,2022-09-27,1992.837874
1,0,2022-09-28,1993.026917
2,0,2022-09-29,1981.524299
3,1,2022-09-27,2993.408144
4,1,2022-09-28,2993.582053
5,1,2022-09-29,2982.066608
6,2,2022-09-27,3993.473395
7,2,2022-09-28,3993.639452
8,2,2022-09-29,3982.13657
9,3,2022-09-27,4993.459173


## Backtest validation of pipeline

In [18]:
ids, test, pred, fit_time, forecast_time = strategy.back_test(dataset, cv=1)

freq: Day; period: 1


0:	learn: 0.9618043	test: 0.9656878	best: 0.9656878 (0)	total: 5.26ms	remaining: 5.25s
500:	learn: 0.0051787	test: 0.0052308	best: 0.0052308 (500)	total: 1.27s	remaining: 1.27s
999:	learn: 0.0030887	test: 0.0032962	best: 0.0032962 (999)	total: 2.58s	remaining: 0us

bestTest = 0.00329621851
bestIteration = 999

Fold 0:
MultiRMSE: 0.0032962185095815415
0:	learn: 0.9647141	test: 0.9623340	best: 0.9623340 (0)	total: 7.71ms	remaining: 7.71s
500:	learn: 0.0055104	test: 0.0057187	best: 0.0057187 (500)	total: 1.27s	remaining: 1.26s
999:	learn: 0.0033140	test: 0.0035696	best: 0.0035696 (999)	total: 2.55s	remaining: 0us

bestTest = 0.003569589613
bestIteration = 999

Fold 1:
MultiRMSE: 0.003569589613068342
Mean MultiRMSE: 0.0034
Std: 0.0001
freq: Day; period: 1


In [19]:
get_results(cv=1, regime="global", y_true=test, y_pred=pred, ids=ids)

,y_true_1,y_pred_1,id_1
0,1997.0,1993.466612,0
1,1998.0,1993.89183,0
2,1999.0,1994.653188,0
3,2997.0,2994.040063,1
4,2998.0,2994.45766,1
5,2999.0,2995.221471,1
6,3997.0,3994.116753,2
7,3998.0,3994.53536,2
8,3999.0,3995.289903,2
9,4997.0,4994.105357,3


## Working with raw time series' granularity

Time series come in different granularities, from hourly and daily time series to more complex ones such as the end of each quarter.

If the rows do not contain segments that are too short (that are shorter than history + horizon), then `tsururu` will try to extract the row granularity on its own. We currently support the following types:

- Yearly (and YearlyEnd)
- Quarterly (and Quarterly)
- Monthly (and MonthlyEnd)
- Weekly
- Daily
- Hourly
- Minlutely
- Secondly
- Microsecondly

There is also support for compound granularities (10 days, 15 minutes, 32 seconds, etc.). The correctness of the selected granularity can be checked from the output after the `Dataset` class has been created.

However, there are tricky situations (e.g. 28 days) where the monthly granularity may be guessed incorrectly. Therefore, it is possible to set your own granularity using the `pd.DateOffset` class or related classes from `pandas.tseries.offsets`, which must be fed as `delta` parameter into the `Dataset` class. Then the time column will be processed according to the user's settings.

Consider a time series where each point is exactly __28 daily points away__ from each other

In [20]:
df_path_2 = "datasets/global/simulated_data_to_check_28D.csv"

# Configure the features settings
dataset_params_2 = {
    "target": {
        "columns": ["value"],
        "type": "continious",
    },
    "date": {
        "columns": ["date"],
        "type": "datetime",
    },
    "id": {
        "columns": ["id"],
        "type": "categorical",
    }
}

In [21]:
dataset_2 = TSDataset(
    data=pd.read_csv(df_path_2),
    columns_params=dataset_params_2,
)

freq: Month; period: 1.0


We see that the frequency of the series is incorrectly defined as monthly. Let's try to pass the `delta` parameter.

In [22]:
dataset_2 = TSDataset(
    data=pd.read_csv(df_path_2),
    columns_params=dataset_params_2,
    delta=pd.DateOffset(days=28),
)

Custom OffSet: <DateOffset: days=28>


Now it's all detected correctly.